###### init

In [ ]:
# de los resultados generales crea silmulaciones con % de
# datos para que luego sean procesados

In [2]:
# import
from elec_bol20 import *
import elec_bol20.util as ebu
import bokeh.layouts
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import Figure, output_file, show
import bokeh.tile_providers

###### abrir los datos
abrimos los datos del padrón de votación del 2019

In [16]:
p = os.path.join(ebu.DATA_PATH1_2020,'comp/exportacion_EG2020_actual.csv')
df_comp = pd.read_csv(p).set_index('ID_MESA')
co = ['VV', 'BL', 'NU', 'VOTO_EMITIDO','CREEMOS', 'MAS', 'FPV',
       'PAN_BOL', 'CC','NUA']
df_comp = df_comp[co]

In [17]:
df_comp.columns

Index(['VV', 'BL', 'NU', 'VOTO_EMITIDO', 'CREEMOS', 'MAS', 'FPV', 'PAN_BOL',
       'CC', 'NUA'],
      dtype='object')

In [18]:
df_comp['ID_RECI'] = (df_comp.index/100).astype(np.int64)
df_comp['COUNT'] = True

p = os.path.join(ebu.DATA_PATH1_2020,'z010R_geopadron_mesas_2020_ALL.csv')
df_all = pd.read_csv(p).set_index('ID_MESA')
['ID_RECI', 'ID_MESA', 'HAB', 'INHAB']

df_all['VV'] = 0
df_all['COUNT'] = False

p = os.path.join(ebu.DATA_PATH1_2020,'z020_geopadron_recintos_2020_ALL_DEN.csv')
df_den = pd.read_csv(p).set_index('ID_RECI')
['ID_RECI', 'LAT', 'LON', 'HAB', 'INHAB', 'PAIS', 'N_MESAS', 'REC',
 'MUN', 'BOL', 'CIU', 'PROV', 'DEP', 'URB', 'DEN_C', 'DEN']

df_den = df_den[['LAT', 'LON', 'PAIS', 'N_MESAS', 'REC',
                 'MUN', 'BOL', 'CIU', 'PROV', 'DEP', 'URB', 'DEN_C', 'DEN']]

_s  = df_comp.index.isin(df_all.index)
assert (~_s).sum() == 0

b = ~df_all.index.isin(df_comp.index)
df_trim = df_all[b]
assert len(df_all) - len(df_comp) == len(df_trim)
df_concat = pd.concat([df_comp,df_trim])
df_full = df_concat.join(df_den,how='left',on='ID_RECI')
p = os.path.join(ebu.DATA_PATH1_2020,'z030_carto_xy.csv')
df_xy = pd.read_csv(p).set_index('ID_RECI')
df2 = df_full.join(df_xy,on='ID_RECI',how='left')
res = pd.cut(df2['DEN'],ebu.DEN_LIMS,labels=ebu.DEN_CODES)
df2['DEN_CODES'] = res.astype(int)

In [19]:
df2

,VV,BL,NU,VOTO_EMITIDO,CREEMOS,MAS,FPV,PAN_BOL,CC,NUA,...,BOL,CIU,PROV,DEP,URB,DEN_C,DEN,X,Y,DEN_CODES
ID_MESA,,,,,,,,,,,,,,,,,,,,,
110046138042512901,141,0.0,0.0,141.0,2.0,34.0,0.0,1.0,104.0,0.0,...,False,Berlín,Berlín,Berlín,exterior,exterior,2.588192,-54.194191,-8.800484,0
110111036112505505,119,0.0,7.0,126.0,4.0,74.0,2.0,1.0,38.0,0.0,...,False,Buenos Aires,Buenos Aires,Buenos Aires,exterior,exterior,158.343774,-58.477989,-25.861461,1
110111036112505506,129,0.0,3.0,132.0,4.0,83.0,3.0,1.0,38.0,0.0,...,False,Buenos Aires,Buenos Aires,Buenos Aires,exterior,exterior,158.343774,-58.477989,-25.861461,1
110111036112505508,105,0.0,6.0,111.0,5.0,64.0,1.0,0.0,35.0,0.0,...,False,Buenos Aires,Buenos Aires,Buenos Aires,exterior,exterior,158.343774,-58.477989,-25.861461,1
110111036112516203,129,0.0,0.0,129.0,2.0,89.0,5.0,1.0,32.0,0.0,...,False,Buenos Aires,Buenos Aires,Buenos Aires,exterior,exterior,253.322801,-58.561146,-25.976533,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112137738202612003,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,Ginebra,Ginebra,Ginebra,exterior,exterior,13.727062,-54.410626,-8.849485,0
112137738202612004,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,Ginebra,Ginebra,Ginebra,exterior,exterior,13.727062,-54.410626,-8.849485,0
112137738202612005,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,Ginebra,Ginebra,Ginebra,exterior,exterior,13.727062,-54.410626,-8.849485,0


In [20]:
#DEFINICIONES
MYE = -5
MYS = -25
MXE = -50
MXS = -75
CYE = -5
CYS = -25
CXE = -50
CXS = -75
BAR_TITLE = "CC  < diferencia [%] >  MAS"
# PIXELS
FIG_WIDTH = 700
C_BAR_HIGH = 80
C_BAR_LOW = -80
PALETTE = ebu.P_DIF
CART_SLIDER_INIT = .5
FILE_OUT = ebu.DIR+'/htlml_1_intermedios/2020/z060_carto_map_mas_cc.html'

MAP_CIRCLE_SIZE_OFFSET = 5
RATIO_CIRCLE_MAP = 7
RATIO_CIRCLE_CARTO = 500
TOOL_TIP = [
    ('Inscritos', '@HAB'),
    ('PAIS, Municipalidad', '@PAIS, @MUN'),
    ('Recinto', '@REC'),
    ('MAS [%]', '@mas{0.0}'),
    ('CC [%]','@cc{0.0}'),
    ('Diferencia [%]', '@ad_mas_cc{0.0} (@mas_o_cc)'),
    ('------','------')
    # ('DEN %', '@DEN')
    # ('PAIS', '@PAIS'),
]
#



df = ebu.open_combine_2019()
_mean = ['X', 'Y', 'LAT', 'LON', 'DEN', ]
_sum = ['HAB', 'CC', 'MAS', 'PDC', 'VV']
_first = ['PAIS', 'REC', 'MUN', 'BOL']
# agrupamos por recinto
_gr = df.groupby('ID_RECI')
rec_df = _gr[_mean].mean()
rec_df[_sum] = _gr[_sum].sum()
rec_df[_first] = _gr[_first].first()

rec_df['D_MAS_CC'] = rec_df['MAS'] - rec_df['CC']
rec_df['d_mas_cc'] = rec_df['D_MAS_CC'] / rec_df['VV'] * 100
rec_df['r'] = np.sqrt(rec_df['HAB']) / RATIO_CIRCLE_CARTO
rec_df['r2'] = np.sqrt(rec_df['HAB']) / RATIO_CIRCLE_MAP + MAP_CIRCLE_SIZE_OFFSET


res = ebu.lola_to_cart(rec_df['LON'].values, rec_df['LAT'].values)
rec_df['GX'] = res[0]
rec_df['GY'] = res[1]

needed_cols = ['X', 'Y', 'd_mas_cc', 'r', 'LAT', 'LON', 'PAIS', 'REC', 'MUN', 'DEN'
                                                                              'GX', 'GY']

In [21]:
# order by density
rec_df = rec_df.sort_values('DEN', axis=0, ascending=True)

In [22]:
# remove nans
rec_df = rec_df.dropna(axis=0)
assert rec_df.isna().sum().sum() == 0

## Carto MAS - CC

###### código

In [23]:

# output_file(os.path.join(ebu.DATA_FIG_OUT, "carto_map_mas_cc.html"))

In [24]:
# rec_df_spl = rec_df.sample(200).copy()
rec_df_spl = rec_df.copy()

In [25]:
# DATA
# bokeh.plotting.output_notebook()
bokeh.plotting.output_file(FILE_OUT)
cart_init_val = CART_SLIDER_INIT
data = rec_df_spl.copy()
data['x'] = data['LON'] * (1 - cart_init_val) + data['X'] * cart_init_val
data['y'] = data['LAT'] * (1 - cart_init_val) + data['Y'] * cart_init_val

In [26]:
# COLOR

from bokeh.transform import linear_cmap
from bokeh.transform import log_cmap

cm = linear_cmap('d_mas_cc', palette=PALETTE, low=C_BAR_LOW, high=C_BAR_HIGH)
# cm = log_cmap('DEN', palette=bokeh.palettes.Viridis11, low=1, high=10000)

In [27]:
# SOURCES
data['mas'] = data['MAS']/data['VV'] * 100
data['cc'] = data['CC']/data['VV'] * 100
data['ad_mas_cc'] = data['d_mas_cc'].abs()
data['mas_o_cc'] = 'n'
data.loc[data['d_mas_cc']>=0,'mas_o_cc'] = 'MAS'
data.loc[data['d_mas_cc']<0,'mas_o_cc'] = 'CC'

In [28]:

source_master = ColumnDataSource(data)
source_red_map = ColumnDataSource({'gx': [], 'gy': []})
la, lo = ebu.get_la_lo_bolivia()
source_bol = ColumnDataSource({'la': la, 'lo': lo})
# source_red_car = ColumnDataSource({'lo': [], 'la': []})

In [29]:
# JS CODE
code_draw_red_map = """
const data = {'gx': [], 'gy': []}
const indices = cb_data.index.indices
for (var i = 0; i < indices.length; i++ ) {
        data['gx'].push(source_master.data.GX[indices[i]])
        data['gy'].push(source_master.data.GY[indices[i]])
}
source_red_map.data = data
"""

code_draw_red_car = """
const data = {'lo': [], 'la': []}
const indices = cb_data.index.indices
for (var i = 0; i < indices.length; i++) {
        data['lo'].push(source_master.data.x[indices[i]])
        data['la'].push(source_master.data.y[indices[i]])
}
source_red_car.data = data
"""

code_merged = """
const data_map = {'lo': [], 'la': []}
const data_car = {'gx': [], 'gy': []}
const indices = cb_data.index.indices
for (var i = 0; i < indices.length; i++) {
        data_map['lo'].push(source_master.data.x[indices[i]])
        data_map['la'].push(source_master.data.y[indices[i]])
        data_car['gx'].push(source_master.data.GX[indices[i]])
        data_car['gy'].push(source_master.data.GY[indices[i]])
}
source_red_car.data = data_car
source_red_map.data = data_map
"""

code_slider = """
    var data = source.data;
    var f = cb_obj.value
    var x = data['x']
    var y = data['y']
    var Y = data['Y']
    var X = data['X']
    var lat = data['LAT']
    var lon = data['LON']
    for (var i = 0; i < x.length; i++) {
        y[i] = (1-f)*lat[i] + f*Y[i]
        x[i] = (1-f)*lon[i] + f*X[i]
    }
    source.change.emit();
"""

# FIGURES
pw = FIG_WIDTH
cart_fig = Figure(plot_width=pw, plot_height=pw, output_backend="webgl")
map_fig = Figure(plot_width=pw, plot_height=pw,
                 x_axis_type='mercator',
                 y_axis_type='mercator',
                 output_backend="webgl",
                 )
cart_fig.background_fill_color = "grey"
cart_fig.background_fill_alpha = .5
# cb_fig = bokeh.plotting.Figure(plot_height=pw,plot_width=)
# cb_fig.toolbar.logo = None
# cb_fig.toolbar_location = None

In [ ]:
# SCATTER

# noinspection PyUnresolvedReferences
# add tiles
tile_provider = bokeh.tile_providers.get_provider(
    bokeh.tile_providers.Vendors.CARTODBPOSITRON)
map_fig.add_tile(tile_provider)

# scatter in map
map_fig.scatter(
    'GX', 'GY', source=source_master, size='r2',
    color=cm
)

#todo if we wont use map then we nee to delete the source
# cart_fig.line('lo', 'la', source=source_bol, color='black')
cart_fig.scatter('x', 'y', source=source_master, radius='r',
                 color=cm
                 )

red_scat_map = map_fig.circle_cross('gx', 'gy',
                                    source=source_red_map,
                                    #                                color='red',
                                    fill_color=None,
                                    #                                line_color='green',
                                    size=20,
                                    line_color="white",
                                    line_width=4
                                    )

red_scat_map = map_fig.circle_cross('gx', 'gy',
                                    source=source_red_map,
                                    #                                color='red',
                                    fill_color=None,
                                    #                                line_color='green',
                                    size=20,
                                    line_color="red",
                                    line_width=1
                                    )
# red_scat_car = cart_fig.scatter('lo', 'la',
# source=source_red_car, color='green')

# add a hover tool that sets the link data for a hovered circle

# callbacks
callback_red_map = CustomJS(
    args={'source_master': source_master,
          'source_red_map': source_red_map,
          # 'source_red_car':source_red_car
          },
    code=code_draw_red_map)
# code = code_merged)

# callback_red_car = CustomJS(
#     args={'source_master': source_master, 'source_red_car': source_red_car},
#     code=code_draw_red_car)

# tools



hover_cart = bokeh.models.HoverTool(
    tooltips=TOOL_TIP,
    callback=callback_red_map,
    # renderers = [red_scat_car]

)
cart_fig.add_tools(hover_cart, )

hover_map = bokeh.models.HoverTool(
    tooltips=TOOL_TIP,
    # callback=callback_red_car,
    # renderers = [red_scat_map]
)
map_fig.add_tools(hover_map, )

# slider
callback_slider = CustomJS(args=dict(source=source_master),
                           code=code_slider)

slider = Slider(start=0, end=1, value=cart_init_val, step=.01, title="carto")
slider.js_on_change('value', callback_slider)

In [ ]:
# COLOR BAR
ml = {int(i):str(np.abs(i)) for i in np.arange(-80,81,20)}
cb = bokeh.models.ColorBar(
    color_mapper=cm['transform'],
    width=int(.9*FIG_WIDTH),
    location=(0, 0),
    #     title="DEN (N/km^2)",
    # title=(BAR_TITLE),
    # margin=0,padding=0,
    title_standoff=10,
    # ticker=bokeh.models.LogTicker(),
    orientation='horizontal',
    major_label_overrides=ml


)

cart_fig.add_layout(cb, 'above')
# cb.title_text_align = 'left'
cart_fig.title.text=BAR_TITLE
cart_fig.title.align='center'

# layout = row(column(slider, cart_f),map_f)
layout = bokeh.layouts.gridplot(
    [[slider, None], [cart_fig, map_fig]]
    , merge_tools=False,
    sizing_mode='scale_width'
)
layout.width = 800
# layout = bokeh.layouts.column([slider, cart_fig])

cart_fig.x_range.start = CXS
cart_fig.x_range.end = CXE
cart_fig.y_range.start = CYS
cart_fig.y_range.end = CYE

_ll = ebu.lola_to_cart(lo=[MXS, MXE], la=[MYS, MYE])
map_fig.x_range.start = _ll[0][0]
map_fig.x_range.end = _ll[0][1]
map_fig.y_range.start = _ll[1][0]
map_fig.y_range.end = _ll[1][1]

cart_fig.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
cart_fig.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
cart_fig.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
cart_fig.yaxis.minor_tick_line_color = None
cart_fig.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels
cart_fig.yaxis.major_label_text_font_size = '0pt'  # turn off y-axis tick labels

###### gráfica

Y también podemos hacer correspondencia entre el cartograma (derecha) y el mapa real(izquierda)

In [ ]:
bokeh.plotting.show(layout)